In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 迁移：使用预构建的训练容器的自定义 Scikit-Learn 模型

<table align="left">
  <td>
<a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-custom-scikit-learn-prebuilt-container.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-custom-scikit-learn-prebuilt-container.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
     <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-custom-scikit-learn-prebuilt-container.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>         
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex AI SDK来训练和部署自定义的表格分类scikit-learn模型，用于批量预测。

了解更多关于[迁移到Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai)和[自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

### 目标

在本教程中，您将学习如何使用 `Vertex AI Training` 创建一个自定义训练模型，并使用 `Vertex AI Batch Prediction` 对训练模型进行批量预测。

您将学习如何使用 Vertex AI SDK for Python 在 Docker 容器中的 Python 脚本中创建一个自定义训练模型，然后通过发送数据对部署的模型进行预测。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Training`
- `Vertex AI Batch Prediction`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

所执行的步骤包括：

- 为训练一个 scikit-learn 模型创建一个 `Vertex AI` 自定义作业。
- 将训练好的模型工件上传为 `Model` 资源。
- 进行批量预测。
- 将模型部署到端点。
- 进行在线预测。

### 数据集

本教程使用的数据集是UCI机器学习【美国人口普查数据（1990年）数据集】(https://archive.ics.uci.edu/ml/datasets/US+Census+Data+(1990))。您在本教程中使用的数据集版本存储在公共云存储桶中。

该数据集预测一个人的收入是否超过5万美元。

### 费用

本教程使用了 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 的定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成费用估算。

## 安装

安装以下所需的包以执行这个笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 tensorflow

只有 Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

开始之前

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您在进行实况教程会话中，您可能使用共享的测试账户或项目。为避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的谷歌云账户

根据您的Jupyter环境，您可能需要手动验证。请按照以下相关说明进行操作。

**1. Vertex AI工作台**
* 无需操作，因为您已经验证通过。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上为您的服务帐户授予云存储权限。

### 创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

导入库并定义常量

In [ ]:
import json
import os

import google.cloud.aiplatform as aip
import tensorflow as tf

初始化Python的Vertex SDK
为您的项目和对应的存储桶初始化Python的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置预构建容器

设置用于训练和预测的预构建Docker容器镜像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TRAIN_VERSION = "scikit-learn-cpu.0-23"
DEPLOY_VERSION = "sklearn-cpu.1-0"

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE)
print("Deployment:", DEPLOY_IMAGE)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存。
     - `n1-highcpu`：每个vCPU 0.9GB的内存。
 - `vCPUs`：\[2、4、8、16、32、64、96\]个

*注：以下类型不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注：您也可以使用n2和e2机器类型来进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

### 检查培训包

#### 包布局

在开始培训之前，您可以查看 Python 包是如何为自定义训练任务组装的。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是用于将包安装到 Docker 镜像的操作环境中的说明。

文件 `trainer/task.py` 是执行自定义训练任务的 Python 脚本。*注意*，在工作池规范中引用时，我们将目录斜杠替换为点 (`trainer.task`) 并删除文件后缀 (`.py`)。

#### 包装配

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: US Census Data (1990) tabular binary classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

In [ ]:
%%writefile custom/trainer/task.py
# Single Instance Training for Census Income

from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer
import datetime
import pandas as pd

from google.cloud import storage

import numpy as np
import argparse
import os
import sys

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
args = parser.parse_args()
print('Python Version = {}'.format(sys.version))

# Public bucket holding the census data
bucket = storage.Client().bucket('cloud-samples-data')

# Path to the data inside the public bucket
blob = bucket.blob('ai-platform/sklearn/census_data/adult.data')
# Download the data
blob.download_to_filename('adult.data')

# Define the format of your input data including unused columns (These are the columns from the census data files)
COLUMNS = (
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income-level'
)



# Categorical columns are columns that need to be turned into a numerical value to be used by scikit-learn
CATEGORICAL_COLUMNS = (
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country'
)

# Load the training census dataset
with open('./adult.data', 'r') as train_data:
    raw_training_data = pd.read_csv(train_data, header=None, names=COLUMNS)

# Remove the column we are trying to predict ('income-level') from our features list
# Convert the Dataframe to a lists of lists
train_features = raw_training_data.drop('income-level', axis=1).values.tolist()
# Create our training labels list, convert the Dataframe to a lists of lists
train_labels = (raw_training_data['income-level'] == ' >50K').values.tolist()

# Since the census data set has categorical features, we need to convert
# them to numerical values. We'll use a list of pipelines to convert each
# categorical column and then use FeatureUnion to combine them before calling
# the RandomForestClassifier.
categorical_pipelines = []

# Each categorical column needs to be extracted individually and converted to a numerical value.
# To do this, each categorical column will use a pipeline that extracts one feature column via
# SelectKBest(k=1) and a LabelBinarizer() to convert the categorical value to a numerical one.
# A scores array (created below) will select and extract the feature column. The scores array is
# created by iterating over the COLUMNS and checking if it is a CATEGORICAL_COLUMN.
for i, col in enumerate(COLUMNS[:-1]):
    if col in CATEGORICAL_COLUMNS:
        # Create a scores array to get the individual categorical column.
        # Example:
        #  data = [39, 'State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical',
        #         'Not-in-family', 'White', 'Male', 2174, 0, 40, 'United-States']
        #  scores = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        #
        # Returns: [['State-gov']]
        # Build the scores array.
        scores = [0] * len(COLUMNS[:-1])
        # This column is the categorical column we want to extract.
        scores[i] = 1
        skb = SelectKBest(k=1)
        skb.scores_ = scores
        # Convert the categorical column to a numerical value
        lbn = LabelBinarizer()
        r = skb.transform(train_features)
        lbn.fit(r)
        # Create the pipeline to extract the categorical feature
        categorical_pipelines.append(
            ('categorical-{}'.format(i), Pipeline([
                ('SKB-{}'.format(i), skb),
                ('LBN-{}'.format(i), lbn)])))

# Create pipeline to extract the numerical features
skb = SelectKBest(k=6)
# From COLUMNS use the features that are numerical
skb.scores_ = [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0]
categorical_pipelines.append(('numerical', skb))

# Combine all the features using FeatureUnion
preprocess = FeatureUnion(categorical_pipelines)

# Create the classifier
classifier = RandomForestClassifier()

# Transform the features and fit them to the classifier
classifier.fit(preprocess.transform(train_features), train_labels)

# Create the overall model as a single pipeline
pipeline = Pipeline([
    ('union', preprocess),
    ('classifier', classifier)
])

# Split path into bucket and subdirectory
bucket = args.model_dir.split('/')[2]
subdirs = args.model_dir.split('/')[3:]
subdir = subdirs[0]
subdirs.pop(0)



for comp in subdirs:
    subdir = os.path.join(subdir, comp)

# Write model to a local file
joblib.dump(pipeline, 'model.joblib')

# Upload the model to GCS
bucket = storage.Client().bucket(bucket)
blob = bucket.blob(subdir + 'model.joblib')
blob.upload_from_filename('model.joblib')

#### 将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的tar文件，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_census.tar.gz

训练一个模型

### [training.create-python-pre-built-container](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container) 

### [training.create-python-pre-built-container](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)

### 创建并运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：1) 创建一个自定义训练任务，2) 运行这个任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，具有以下参数：

- `display_name`: 自定义训练任务的可读名称。
- `container_uri`: 训练容器镜像。
- `requirements`: 训练容器镜像的包要求（例如，pandas）。
- `script_path`: 训练脚本的相对路径。

In [ ]:
job = aip.CustomTrainingJob(
    display_name="census_" + UUID,
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs", "tensorflow-datasets"],
)

print(job)

示例输出：

<google.cloud.aiplatform.training_jobs.CustomTrainingJob object at 0x7feab1346710>

运行定制的训练作业

接下来，通过调用`run`方法并使用以下参数来运行自定义作业以启动训练作业：

- `replica_count`：用于训练的计算实例数量（replica_count = 1表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `base_output_dir`：将模型工件写入的Cloud Storage位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, UUID)


job.run(
    replica_count=1, machine_type=TRAIN_COMPUTE, base_output_dir=MODEL_DIR, sync=True
)

MODEL_DIR = MODEL_DIR + "/model"
model_path_to_deploy = MODEL_DIR

### [general.import-model]（https://cloud.google.com/vertex-ai/docs/general/import-model）

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到`Model`资源，使用以下参数：

- `display_name`: `Model`资源的可读名称。
- `artifact`: 训练模型工件的云存储位置。
- `serving_container_image_uri`: 用于服务的容器映像。
- `sync`: 是否异步或同步执行上传。

如果`upload()`方法以异步方式运行，您可以随后使用`wait()`方法阻塞直至完成。

In [ ]:
model = aip.Model.upload(
    display_name="census_" + UUID,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=False,
)

model.wait()

示例输出：

    信息：google.cloud.aiplatform.models：创建模型
    信息：google.cloud.aiplatform.models：创建模型的LRO：projects/759209241365/locations/us-central1/models/925164267982815232/operations/3458372263047331840
    信息：google.cloud.aiplatform.models：模型已创建。资源名称：projects/759209241365/locations/us-central1/models/925164267982815232
    信息：google.cloud.aiplatform.models：要在另一个会话中使用此模型：
    信息：google.cloud.aiplatform.models：model = aiplatform.Model('projects/759209241365/locations/us-central1/models/925164267982815232')

进行批量预测

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### [预测.批量预测](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

制作测试项目

您可以使用合成数据作为测试数据项目。不要担心我们使用合成数据 - 我们只是想展示如何进行预测。

In [ ]:
INSTANCES = [
    [
        25,
        "Private",
        226802,
        "11th",
        7,
        "Never-married",
        "Machine-op-inspct",
        "Own-child",
        "Black",
        "Male",
        0,
        0,
        40,
        "United-States",
    ],
    [
        38,
        "Private",
        89814,
        "HS-grad",
        9,
        "Married-civ-spouse",
        "Farming-fishing",
        "Husband",
        "White",
        "Male",
        0,
        0,
        50,
        "United-States",
    ],
]

### 生成批量输入文件

现在生成一个批量输入文件，将其存储在您的本地云存储桶中。预测请求中的每个实例都是一个列表，如下所示：

[ [内容_1], [内容_2] ]

- `内容`：测试项的特征值列表。

In [ ]:
gcs_input_uri = BUCKET_URI + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for i in INSTANCES:
        f.write(json.dumps(i) + "\n")

! gsutil cat $gcs_input_uri

### 发送批量预测请求

现在，您的模型资源经过训练，您可以通过调用`batch_predict()`方法来进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`：用于训练的机器类型。
- `sync`：如果设置为True，则调用将在等待异步批处理作业完成时阻塞。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name="census_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    predictions_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=False,
)

print(batch_predict_job)

示例输出：

    信息：google.cloud.aiplatform.jobs:正在创建BatchPredictionJob
    <google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0>正在等待上游依赖项完成。
    信息：google.cloud.aiplatform.jobs：已创建BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
    信息：google.cloud.aiplatform.jobs：要在另一个会话中使用此BatchPredictionJob：
    信息：google.cloud.aiplatform.jobs：bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296')
    信息：google.cloud.aiplatform.jobs：查看Batch Prediction Job：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296当前状态：
    JobState.JOB_STATE_RUNNING

等待批量预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

*示例输出：*

    INFO：google.cloud.aiplatform.jobs：BatchPredictionJob已创建。 资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
    INFO：google.cloud.aiplatform.jobs：在另一个会话中使用此BatchPredictionJob：
    INFO：google.cloud.aiplatform.jobs：bpj = aiplatform.BatchPredictionJob（'projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328'）
    INFO：google.cloud.aiplatform.jobs：查看批量预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
    INFO：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    INFO：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    ……（重复多次）
    INFO：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_SUCCEEDED
    INFO：google.cloud.aiplatform.jobs：批量预测作业已完成运行。 资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

获得预测

接下来，从已完成的批量预测作业中获取结果。

结果被写入到您在批量预测请求中指定的Cloud Storage输出存储桶中。您可以调用iter_outputs()方法获取生成结果的每个Cloud Storage文件的列表。每个文件都以JSON格式包含一个或多个预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)

{'instance': [25, 'Private', 226802, '11th', 7, 'Never-married', 'Machine-op-inspct', 'Own-child', 'Black', 'Male', 0, 0, 40, 'United-States'], 'prediction': False}

示例输出: 

{'实例': [25, '私人', 226802, '11th', 7, '未婚', '机器操作检查员', '自己的孩子', '黑色', '男性', 0, 0, 40, '美国'], '预测': False}

## 进行在线预测

### [predictions.deploy-model-api](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

### [predictions.deploy-model-api](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

## 部署模型

接下来，为在线预测部署您的模型。要部署模型，您需要调用`deploy`方法，并提供以下参数：

- `deployed_model_display_name`：部署模型的人类可读名称。
- `traffic_split`：在端点上流量的百分比，流向该模型，以字典形式指定一个或多个键值对。
如果只有一个模型，则指定为{ "0": 100 }，其中 "0" 指的是上传的这个模型，100 表示100%的流量。
如果端点上存在其他模型，需要对流量进行分配，则使用model_id指定为{ "0": percent, model_id: percent, ... }，其中model_id是已部署在端点的现有模型的模型ID。各比例之和必须为100。
- `machine_type`：用于训练的机器类型。
- `starting_replica_count`：最初预留的计算实例数。
- `max_replica_count`：扩展至的最大计算实例数。在本教程中，只预留了一个实例。

In [ ]:
DEPLOYED_NAME = "census-" + UUID

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

INFO：google.cloud.aiplatform.models：创建端点
INFO：google.cloud.aiplatform.models：创建端点后台LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/4087251132693348352
INFO：google.cloud.aiplatform.models：端点已创建。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
INFO：google.cloud.aiplatform.models：要在另一个会话中使用此端点：
INFO：google.cloud.aiplatform.models：endpoint = aiplatform.Endpoint('projects/759209241365/locations/us-central1/endpoints/4867177336350441472')
INFO：google.cloud.aiplatform.models：部署模型到端点：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
INFO：google.cloud.aiplatform.models：部署端点模型后台LRO：projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/1691336130932244480
INFO：google.cloud.aiplatform.models：端点模型已部署。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472

制作测试项目

您可以使用合成数据作为测试数据项目。请不要担心我们使用合成数据 - 我们只是想展示如何进行预测。

In [ ]:
INSTANCE = [
    25,
    "Private",
    226802,
    "11th",
    7,
    "Never-married",
    "Machine-op-inspct",
    "Own-child",
    "Black",
    "Male",
    0,
    0,
    40,
    "United-States",
]

### 进行预测

现在您的 `Model` 资源已部署到一个 `Endpoint` 资源上，您可以通过向 `Endpoint` 资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式是：

    [feature_list]

由于 predict() 方法可以接受多个项（实例），请将您的单个测试项作为一个测试项列表发送。

#### 响应

从 predict() 调用的响应是一个包含以下条目的 Python 字典：

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `predictions`: 每个类别标签的预测置信度，介于 0 和 1 之间。
- `deployed_model_id`: 执行预测的 Vertex AI `Model` 资源的标识符。

In [ ]:
instances_list = [INSTANCE]

prediction = endpoint.predict(instances_list)
print(prediction)

示例输出：

    预测（predictions=[False]，deployed_model_id='7220545636163125248'，explanations=None）

## 撤销模型

当您完成预测时，您需要从“端点”资源中撤销模型。这将取消所有计算资源，并结束部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 模型
- 终端
- AutoML训练作业
- 批处理作业
- 云存储桶

In [ ]:
# delete endpoint
endpoint.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI